# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [4]:
# Load the stock prices
df = pd.read_csv('Resources/MSR_sp500_5yr_close.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
#df = df["chosen stocks"]
df.drop(columns=['date.1','date.2'], inplace=True)
df.head()

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-04-06 00:00:00+00:00,165.48,46.30,57.61,70.87,95.69,22.670,76.23,2.70,145.73,13.33,...,NaN,130.72,35.05,12.95,41.990,36.07,79.73,117.36,26.900,46.33
2015-04-07 00:00:00+00:00,166.21,46.19,58.42,69.73,96.39,22.770,75.65,2.70,146.71,13.14,...,NaN,127.88,34.51,12.96,42.330,36.07,78.29,116.01,27.210,46.36
2015-04-08 00:00:00+00:00,166.27,46.61,58.81,69.63,95.12,22.960,75.44,2.71,148.81,13.12,...,NaN,129.28,34.43,13.10,42.550,35.25,78.30,116.12,27.270,46.21
2015-04-09 00:00:00+00:00,166.79,47.00,60.32,69.80,95.47,23.000,76.33,2.72,148.05,13.05,...,NaN,136.63,34.32,13.13,43.325,34.96,78.92,117.20,27.535,46.55
2015-04-10 00:00:00+00:00,167.07,47.15,62.00,69.35,94.69,22.925,76.53,2.76,150.20,13.20,...,NaN,136.02,34.67,13.26,43.830,35.20,80.02,117.45,27.330,47.17


In [5]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window + 1), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [6]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Try a window size anywhere from 1 to 10 and see how the model performance changes
#jjp - need to add a for loop 

# Creating the features (X) and target (y) data using the window_data() function.
window_size = 1

# Column index 1 is the `Close` column
feature_column = 5
target_column = 5
X, y = window_data(df, window_size, feature_column, target_column)

In [7]:
# Use 70% of the data for training and the remaineder for testing
# YOUR CODE HERE!
from sklearn.model_selection import train_test_split

split = int(0.7 *len(X))
X_train_rnn = X[: split -1]
X_test = X[split:]
y_train_rnn = y[: split -1]
y_test = y[split:]

X_train_rnn, X_val_rnn, y_train_rnn, y_val_rnn = train_test_split(X_train_rnn, y_train_rnn)

In [8]:
# Use MinMaxScaler to scale the data between 0 and 1. 
# YOUR CODE HERE!
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train_rnn =scaler.transform(X_train_rnn)
X_val_rnn =scaler.transform(X_val_rnn)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train_rnn = scaler.transform(y_train_rnn)
y_val_rnn = scaler.transform(y_val_rnn)
y_test =scaler.transform(y_test)

In [9]:
# Reshape the features for the model
# YOUR CODE HERE!
X_train_rnn = X_train_rnn.reshape((X_train_rnn.shape[0], X_train_rnn.shape[1], 1))
X_val_rnn = X_val_rnn.reshape((X_val_rnn.shape[0], X_val_rnn.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train_rnn[:5]} \n")
print (f"X_val_rnn sample values:\n{X_val_rnn[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.84897892]
  [0.83503626]]

 [[0.95289855]
  [0.97000659]]

 [[0.6386693 ]
  [0.67996045]]

 [[0.44795784]
  [0.46423863]]

 [[0.        ]
  [0.00098879]]] 

X_val_rnn sample values:
[[[0.00543478]
  [0.00354318]]

 [[0.47249671]
  [0.48121292]]

 [[0.1780303 ]
  [0.20517469]]

 [[0.84453228]
  [0.85003296]]

 [[0.81175889]
  [0.83816744]]] 

X_test sample values:
[[[0.99818841]
  [1.        ]]

 [[1.        ]
  [0.99390244]]

 [[0.99390646]
  [0.95006592]]

 [[0.95009881]
  [0.9373764 ]]

 [[0.93741765]
  [0.90507581]]]


---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [11]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# YOUR CODE HERE!
model = Sequential()

number_units = 50
dropout_fraction = 0.7

#first layer:
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train_rnn.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
#second layer
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
#third layer
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
#fourth layer
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
#output layer
model.add(Dense(1))

In [12]:
# Compile the model
# YOUR CODE HERE!
model.compile(optimizer="adam", loss="mean_squared_error")

In [13]:
# Summarize the model
# YOUR CODE HERE!
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 50)             10400     
_________________________________________________________________
dropout (Dropout)            (None, 2, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 50)             20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 50)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 50)             20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 50)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [14]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!
batch_size = 60
epochs = 250
model.fit(
    X_train_rnn,
    y_train_rnn,
    validation_data=(X_val_rnn, y_val_rnn),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

Train on 659 samples, validate on 220 samples
Epoch 1/250
659/659 [==============================] - 6s 10ms/sample - loss: 0.2284 - val_loss: 0.2269
Epoch 2/250
659/659 [==============================] - 0s 179us/sample - loss: 0.1656 - val_loss: 0.1448
Epoch 3/250
659/659 [==============================] - 0s 151us/sample - loss: 0.0926 - val_loss: 0.0680
Epoch 4/250
659/659 [==============================] - 0s 156us/sample - loss: 0.0716 - val_loss: 0.0595
Epoch 5/250
659/659 [==============================] - 0s 148us/sample - loss: 0.0638 - val_loss: 0.0579
Epoch 6/250
659/659 [==============================] - 0s 134us/sample - loss: 0.0538 - val_loss: 0.0346
Epoch 7/250
659/659 [==============================] - 0s 142us/sample - loss: 0.0420 - val_loss: 0.0206
Epoch 8/250
659/659 [==============================] - 0s 156us/sample - loss: 0.0348 - val_loss: 0.0087
Epoch 9/250
659/659 [==============================] - 0s 131us/sample - loss: 0.0308 - val_loss: 0.0114
Epoch 10/2

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [15]:
# Evaluate the model
# YOUR CODE HERE!
model.evaluate(X_test, y_test)

378/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.000612956373188546

In [20]:
seq = df[df.columns[feature_column]][-12:-10].tolist()
forecast=[]
for i in range (5):
    prediction = model.predict([np.array(seq).reshape(1,X_train_rnn.shape[1],1)])
    seq.append(prediction[0][0])
    seq.pop(0)
    forecast.append(prediction[0][0])

In [22]:
forecast

[0.6604551, 0.6605061, 0.66350484, 0.6642647, 0.6665952]

In [16]:
# Make some predictions
# YOUR CODE HERE!
predicted = model.predict(X_test)

In [17]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [18]:
# Create a DataFrame of Real and Predicted values
stock_closing = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stock_closing.head()

,Real,Predicted
0,83.39,73.856308
1,83.02,73.870476
2,80.36,73.885033
3,79.59,73.551163
4,77.63,73.362984


In [19]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!
stock_closing.plot(title='Close price predictions using LSTM RNN on closing price',figsize=(20,10))